In [ ]:
!python3 -m pip install PyMySQL
!python3 -m pip install SQLAlchemy
!python3 -m pip install google-cloud-storage
!python3 -m pip install --upgrade --quiet scikit-sound
!python3 -m pip install --upgrade --quiet pygame
!sudo apt-get -y install ffmpeg
!sudo apt-get -y install python3-pymysql

In [26]:
import sqlalchemy
import numpy as np

from google.cloud import storage
from numpy.fft import fft, ifft
from sksound.sounds import Sound

from tensorflow.keras.models import load_model

In [27]:
class StreamSpectrogram:
    
    def __init__(self, filename, win=32):
        sound = Sound(filename)        
        self.batch = sound.rate * 60
        self.data  = sound.data
        if len(self.data.shape) > 1:
            self.data = self.data[:, 0]    
        self.win = win
        self.filter = filter
        
    def __iter__(self):
        self.i = 0
        return self

    def __next__(self):
        if (self.i + 1) * self.batch < len(self.data): 
            start = self.i       * self.batch
            stop  = (self.i + 1) * self.batch
            raw   = self.data[start:stop]
            spec  = fwd_spectrogram(raw) 
            t,d   = spec.shape
            current = []        
            for i in range(self.win, t, self.win // 2):
                x      = np.reshape(spec[i - self.win:i], (self.win, d, 1))
                mu     = np.mean(x)
                std    = np.std(x) + 1.0
                window = (x - mu) / std
                current.append(window)
            self.i += 1
            return np.stack(current)
        else:
            raise StopIteration
        
def fwd_spectrogram(audio, win=512, step=256):
    '''
    Compute the spectrogram of audio data

    audio: one channel audio
    win: window size for dft sliding window
    step: step size for dft sliding windo
    '''
    spectrogram = []
    hanning = np.hanning(win)
    for i in range(win, len(audio), step):
        start = win // 2
        dft = np.abs(fft(audio[i - win: i] * hanning))[start:win]
        spectrogram.append(dft)
    return np.array(spectrogram)

In [28]:
# PASSWORDS AND STUFF HERE

In [29]:
settings = {
   'user': db_user,
   'pass': db_password,
   'host': host,
     'db': db_name
}
url = 'mysql+pymysql://{user}:{pass}@{host}/{db}'.format(**settings)  # 5432 is the default port
db = sqlalchemy.create_engine(url)

In [30]:
def run_query(query):
    with db.connect() as conn:
        rows = []
        for row in conn.execute(query).fetchall():
            rows.append(dict(row.items()))
        return rows

In [31]:
files = run_query("""
    SELECT 
        x.encoding, y.year, x.filename 
    FROM 
        wdp_ds.audio x 
    JOIN wdp_ds.encoding y ON x.encoding = y.encoding;
""")

In [32]:
paths = ["audio_files/{}/{}".format(file['year'], file['filename']) for file in files]

In [33]:
max_id = run_query("""
    SELECT 
        max(id)
    FROM 
        wdp_ds.not_silent
    """)
if max_id[0]['max(id)'] is None:
    max_id = 0
else:
    max_id = max_id[0]['max(id)']
print(max_id)

2113


In [22]:
noise_classifier = load_model('sil.h5')
client = storage.Client.from_service_account_json('secret.json')
bucket = client.get_bucket('wdp-data')

In [ ]:
id = max_id + 1
for path, file_dict in zip(paths, files):
    print(path, file_dict)
    with open("/tmp/audio.m4a", "wb") as file_obj:
        blob = bucket.blob(path)
        blob.download_to_file(file_obj)
    stream = StreamSpectrogram("/tmp/audio.m4a")
    not_noise = []
    for x in stream:
        y = noise_classifier.predict(x).flatten()
        not_noise.extend([int(np.round(sample)) == 0 for sample in y])

    regions = []
    for i in range(0, len(not_noise)):
        if not_noise[i]:
            start = i * 16 * 256 
            stop  = (i + 1) * 16 * 256 
            if len(regions) > 0: 
                last  = regions[-1]
                if start - last[1] < 48000 * 0.75:
                    start       = regions[-1][0]
                    regions[-1] = (start, stop)
                else:
                    regions.append((start, stop))
            else:
                regions.append((start, stop))
    regions = [(start, stop) for start, stop in regions if stop - start > (16 * 256)]
    if len(regions) > 0:
        with db.connect() as conn:
            for start, stop in regions:                
                conn.execute("INSERT INTO wdp_ds.not_silent VALUES ({}, {}, '{}', {}, {})".format(id, file_dict['encoding'], file_dict['filename'], start, stop))  
                id += 1
    print(file_dict['encoding'], ": ", regions)